In [114]:
import pandas as pd
import datetime
import networkx as nx
from math import radians, cos, sin, asin, sqrt


In [115]:
# lecture des donnees gtfs

df_stops_bus = pd.read_csv('gtfs_bus/stops.txt')
df_routes_bus = pd.read_csv('gtfs_bus/routes.txt')
df_trips_bus = pd.read_csv('gtfs_bus/trips.txt')
df_stop_times_bus = pd.read_csv('gtfs_bus/stop_times.txt')

df_stops_tram = pd.read_csv('gtfs_tram/stops.txt')
df_routes_tram = pd.read_csv('gtfs_tram/routes.txt')
df_trips_tram = pd.read_csv('gtfs_tram/trips.txt')
df_stop_times_tram = pd.read_csv('gtfs_tram/stop_times.txt')

In [131]:

def edges_build(stop_ids_list):
    """
    Builds a sequence of edges in the network appropriate format from a sequence of ids
    @stop_ids_lis : sequence of ids
    @Return       : a sequence of 2-tuples 
    """

    for x in stop_ids_list:
        x = str(x)
    l = len(stop_ids_list)-1
    i = 0
    edges_list = list()
    while (i<l):
        edges_list.append((str(stop_ids_list[i]), str(stop_ids_list[i+1])))
        i = i+1
    return edges_list

def remove_stops_by_id(g,stops_list):
    """"
    Creates a new graph from a former one after removing the given list of ids and the edges involving them
    @g          : the initial graph
    @stops_list : the nodes to remove
    @Return     : the new graph
    """
    
    new_graph=g.copy()
    new_graph.remove_nodes_from(map(str, stops_list))
    return new_graph

def graph_weights(graph, nodes_list):
    weights = list()
    alternatif_path = list()
    i=0
    l=len(nodes_list)
    while(i<l-1):
        weights.append(trips[graph[nodes_list[i]][nodes_list[i+1]]["trip"]].route_id)
        i+=1
    i=0
    j=0
    while(i<l-1):
        alternatif_path.append(nodes_list[i])
        alternatif_path.append(weights[i])
        i+=1
    alternatif_path.append(nodes_list[l-1])
    return alternatif_path


def alternatif_path(graph, src, dst):
    src = str(src)
    dst = str(dst)
    return map(int, graph_weights(graph, nx.shortest_path(graph, source=src, target=dst)))



def breakdown_new_graph(g, trips,stop_1, stop_2, route_id=-1):
    """
    Creates a new graph from a former one with taking in account a breakdown
    @g        : the initial graph
    @trips    : the trips ids
    @stop_1   : one of the extremity points of the breakdown
    @stop_2   : the other extremity
    @route_id : the route id
    @Return   : the new graph
    """
    affected_trips = set()
    for trip in trips:
        if stop_1 in trips[trip].path:
            if stop_2 in trips[trip].path:
                if (trips[trip].route_id == route_id):
                    affected_trips.add(trip)
    new_g = g.copy()
    for edge in new_g.edges():
        if g[edge[0]][edge[1]]["trip"] in affected_trips:
            index_a = trips[g[edge[0]][edge[1]]["trip"]].path.index(edge[0])
            index_b = trips[g[edge[0]][edge[1]]["trip"]].path.index(edge[1])
            index_1 = trips[g[edge[0]][edge[1]]["trip"]].path.index(stop_1)
            index_2 = trips[g[edge[0]][edge[1]]["trip"]].path.index(stop_2)
            if (is_in_order(index_1, index_a, index_2) & is_in_order(index_1, index_b, index_2)):
                new_g.remove_edge(*edge)
    return new_g 


def is_in_order(a,b,c):
    """"
    Tells if a is between b and c
    """

    if ((a<=b & b<=c) | (c<=b & b<=a)):
        return True
    return False



def close_stops(stop_1, stop_2, max_distance):
    """"
    Tells if two stops are close enough according to their GPS coordinates (longitude, latitude)
    @stop_1       : the first stop
    @stop_2       : the other one
    @max_distance : the maximum distance between two "close" stops
    @Return       : True if the distance betwwen the two stops is less than ma_distance. False otherwise
    """
    if(stop_1 == stop_2):
        return False
    if(distance_between_two_stops(stops[stop_1].lon, stops[stop_1].lat, stops[stop_2].lon, stops[stop_2].lat) < max_distance):
        return True
    return False



    
def distance_between_two_stops(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points on the earth (specified in decimal degrees)
    @lon1   : the fisrt point's longitude
    @lat1   : the first point's latitude
    @lon2   : the second point's longitude
    @lat2   : the second point's latitude
    @Return : the harvesine formula
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km     



def route_id_to_name(route_id):
    return df_routes[df_routes["route_id"] == route_id]["route_short_name"].tolist()[0]

def stop_to_route(stop_name):
    stop_id = stop_name_to_id(stop_name)["stop_id"].tolist()[0]
    trip_id = df_stop_times[df_stop_times["stop_id"] == int(stop_id)]["trip_id"].tolist()[0]
    route_id = df_trips[df_trips["trip_id"] == trip_id]["route_id"].tolist()[0]
    return route_id_to_name(route_id)


def stop_id_to_name(stop_id):
    df_tram=df_stops_tram[df_stops_tram["stop_id"] == stop_id]["stop_name"]
    df_bus=df_stops_bus[df_stops_bus["stop_id"] == stop_id]["stop_name"]
    if (len(df_bus)>0):
        return df_bus.tolist()[0]
    if (len(df_tram)>0):
        return df_tram.tolist()[0]
    return "NO_SUCH_ID"


def stop_name_to_id_tram(stop_name):
    return df_stops_tram[df_stops_tram["stop_name"] == stop_name]["stop_id"].tolist()

def stop_name_to_id_bus(stop_name):
    return df_stops_bus[df_stops_bus["stop_name"] == stop_name]["stop_id"].tolist()

def stop_name_to_ids(stop_name):
    ids_list = list ()
    df_tram = df_stops_tram[df_stops_tram["stop_name"] == stop_name]["stop_id"]
    df_bus = df_stops_bus[df_stops_bus["stop_name"] == stop_name]["stop_id"]
    if(len(df_bus)>0):
        ids_list+=df_bus.tolist()
    if(len(df_tram)>0):
        ids_list+=df_tram.tolist()
    return [elt for elt in ids_list if is_integer(elt)]

def stop_names_to_ids(names_list):
    ids_list = list()
    for name in names_list:
        ids_list+=stop_name_to_ids(name)
    return ids_list


def stop_ids_to_names(ids_list):
    return map(stop_id_to_name, ids_list)





def stop_id_to_name_tram(stop_id):
    return df_stops_tram[df_stops_tram["stop_id"] == stop_id]["stop_name"].tolist()[0]

def stop_id_to_name_bus(stop_id):
    return df_stops_bus[df_stops_bus["stop_id"] == stop_id]["stop_name"].tolist()[0]




def is_integer (x):
    return x.isdigit()
    
    
def opposite(stop_1, stop_2):
    if (not stop_id_to_name(stop_1) == stop_id_to_name(stop_2)):
        return False
    trips_list_1 = stops[stop_1].trip_ids
    trips_list_2 = stops[stop_2].trip_ids
    if (len(trips_list_1) == 0 | len(trips_list_2) == 0):
        return False
    for elt in trips_list_1:
        for elt_ in trips_list_2:
            if ((trips[elt].route_id == trips[elt_].route_id) & 
                (trips[elt].direction_id != trips[elt_].direction_id)):
                return True
    return False
    

    

In [117]:
# construction dictionnaire d'arrets


class Stop:
    def __init__(self):
        self.iden = "UNDEFINED"
        self.name = "UNDEFINED"
        self.lon = 1000.0
        self.lat = 1000.0 
        self.trans_type = -1        # 0:Bus, 1:TRAM
        self.trip_ids = list()             # bus/tram line_id

ids = pd.concat([df_stops_bus["stop_id"], df_stops_tram["stop_id"]]).tolist()        
stops = dict()


for elt in ids:
    df_temp_bus = df_stops_bus[df_stops_bus["stop_id"] == elt]
    df_temp_tram = df_stops_tram[df_stops_tram["stop_id"] == elt]
    if(len(df_temp_bus) > 0):
        if(is_integer(df_temp_bus["stop_id"].tolist()[0])):
            stops[elt]=Stop()
            stops[elt].iden = df_temp_bus["stop_id"].tolist()[0]
            stops[elt].lat = df_temp_bus["stop_lat"].tolist()[0]
            stops[elt].lon = df_temp_bus["stop_lon"].tolist()[0] 
            stops[elt].name = df_temp_bus["stop_name"].tolist()[0]
            stops[elt].trans_type = 0
    if(len(df_temp_tram) > 0):
        if(is_integer(df_temp_tram["stop_id"].tolist()[0])):
            stops[elt]=Stop()
            stops[elt].iden = df_temp_tram["stop_id"].tolist()[0]
            stops[elt].lat = df_temp_tram["stop_lat"].tolist()[0]
            stops[elt].lon = df_temp_tram["stop_lon"].tolist()[0] 
            stops[elt].name = df_temp_tram["stop_name"].tolist()[0]
            stops[elt].trans_type = 1

In [118]:
# suppression des repetitions dans trips et stop_times
df_trips_bus = df_trips_bus.drop_duplicates(["route_id", "direction_id", "trip_headsign"] )
df_stop_times_bus = df_stop_times_bus[df_stop_times_bus["trip_id"].isin(df_trips_bus["trip_id"])]
df_trips_tram = df_trips_tram.drop_duplicates(["route_id", "direction_id", "trip_headsign"] )
df_stop_times_tram = df_stop_times_tram[df_stop_times_tram["trip_id"].isin(df_trips_tram["trip_id"])]

In [119]:
#Construction du dictionnaire des trips

class Trip:
    def __init__(self):
        self.iden = "UNDEFINED"
        self.route_id = -2
        self.direction_id = -1
        self.trip_headsign = "UNDEFINED"
        self.route_name = "UNDEFINED"
        self.path = list()
        self.tran = -1

trips = dict()
trips["a_pied"] = Trip()
trips["a_pied"].iden = "a_pied"
trips["a_pied"].route_id = -1
trips["a_pied"].direction_id = -1
trips["a_pied"].trip_headsign = "UNDEFINED"
trips["a_pied"].path = list()

for elt in df_trips_bus["trip_id"]:
    trips[elt] = Trip()
    trips[elt].iden = elt
    trips[elt].route_id = df_trips_bus[df_trips_bus["trip_id"]==elt]["route_id"].tolist()[0]
    trips[elt].direction_id = df_trips_bus[df_trips_bus["trip_id"]==elt]["direction_id"].tolist()[0]
    trips[elt].trip_headsign = df_trips_bus[df_trips_bus["trip_id"]==elt]["trip_headsign"].tolist()[0]
    df_bus = df_stop_times_bus[df_stop_times_bus["trip_id"] == elt]
    trips[elt].path = map(str,df_bus["stop_id"].tolist())
    trips[elt].tran = 0
for elt in df_trips_tram["trip_id"]:
    trips[elt] = Trip()
    trips[elt].iden = elt
    trips[elt].route_id = df_trips_tram[df_trips_tram["trip_id"]==elt]["route_id"].tolist()[0]
    trips[elt].direction_id = df_trips_tram[df_trips_tram["trip_id"]==elt]["direction_id"].tolist()[0]
    trips[elt].trip_headsign = df_trips_tram[df_trips_tram["trip_id"]==elt]["trip_headsign"].tolist()[0]
    df_tram = df_stop_times_tram[df_stop_times_tram["trip_id"] == elt]
    trips[elt].path = map(str,df_tram["stop_id"].tolist())
    trips[elt].tran = 1



In [121]:
# Construction du graphe

g = nx.Graph()
g.clear()
for elt in trips:
    if (not elt == "a_pied"):
        g.add_nodes_from(trips[elt].path)
        g.add_edges_from(edges_build(trips[elt].path), trip=elt)
for elt in g.nodes():
    l = stop_name_to_ids(stops[elt].name)
    for elt_ in l:
        if(g.has_node(elt_) & (not opposite(elt, elt_))):
            g.add_edge(elt,elt_, trip="a_pied")
for elt in g.nodes():
    for elt_ in g.nodes():
        if (close_stops(elt, elt_, 0.1)):
            g.add_edge(elt, elt_, trip="a_pied")
            
            
for stop in g.nodes():
    for trip in trips:
        if stop in trips[trip].path:
            stops[stop].trip_ids.append(trips[trip])
        


In [122]:
g.number_of_nodes()

3368

In [123]:
g.number_of_edges()

10455

In [137]:
new = remove_stops_by_id(g, [3718, 3719, 3757, 3758])
print alternatif_path(new, 3749, 3720)
print stop_name_to_ids("Palais de Justice")
print stop_id_to_name("300")
print stop_id_to_name("3310")

[3749, 60, 3751, 60, 3754, -1, 3801, 15, 823, 15, 3548, 15, 4031, -1, 3720]
['3823', '3858', '4031', '3720', '3721']
Hôpital Pellegrin (Grand Maurian)
Boulevard George V
